In [ ]:
def getR2(m):
    """Return a value for the radius of the secondary given the mass.
    Determined using Wilson & Nordhaus (2019), Eq. 9.
    
    Keyword arguments:
    m -- mass
    """
    import math
    from math import log
    if (m > 0.077):
        r = m**0.92
    elif (m < 0.0026):
        r = 0.10045 # r_jupiter
    else:
        r = 0.117
        r = r - 0.054*(log(m/0.0026)**2)
        r = r + 0.024*(log(m/0.0026)**3)
    return r

def getTInspiral(p, m2):
    """Return an array containing the inspiral time-scale values.
    Does NOT remove the 100 outermost values.
    
    Keyword arguments:
    p -- MESA profile
    m2 -- mass of the secondary
    """
    import mesa_reader as mr
    import matplotlib.pylab as plt
    import numpy as np
    import os
    from math import log
    from scipy.integrate import cumtrapz
    import math

    # import function from another file
    from ipynb.fs.full.functions import getMaxRadiusProfile

    G = 6.67408e-11 # gravitational constant
    # change G to cgs units
    G = G * 1e3

    coreMass = p.he_core_mass + p.c_core_mass + p.o_core_mass + p.si_core_mass + p.fe_core_mass
    # change from Msuns to grams
    coreMass = coreMass*1.989e33

    radius = p.radius
    radius = radius*69.551e9

    # setting up constants
    r2 = getR2(m2)
    m2 = m2*1.989e33 # units
    r2 = r2*69.551e9 # units
    xi = 4
    k = 4 * xi * math.pi * G * m2

    # density
    rho = p.logRho
    rho = 10**rho

    # masses
    masses = p.mass
    masses = masses*1.989e33

    # keplerian velocity
    vkep_r = np.sqrt(G * masses / radius)

    # dM/dr
    dMdr = np.diff(masses) / np.diff(radius)

    # make all the array sizes the same
    vkep_r = vkep_r[:-1]
    radius = radius[:-1]
    rho = rho[:-1]
    masses = masses[:-1]

    # integrand
    integrand = (dMdr - (masses / radius)) * vkep_r / (k * radius * rho)

    # actually integrate
    tInspiral = cumtrapz(y=integrand, x=radius)
    
    return np.flip(tInspiral)

In [ ]:
def plotTInspiral(p, m2, label):
    """Plot the inspiral time scale.
    
    Keyword arguments:
    p -- MESA profile
    m2 -- mass of the secondary
    label -- text appearing in the legend
    """
    import mesa_reader as mr
    import matplotlib.pylab as plt
    import numpy as np
    import os
    from math import log
    from scipy.integrate import cumtrapz
    import math

    # import function from another file
    from ipynb.fs.full.functions import getMaxRadiusProfile

    G = 6.67408e-11 # gravitational constant
    # change G to cgs units
    G = G * 1e3
    
    tInspiral = getTInspiral(p, m2)
    tInspiral = np.flip(tInspiral)

    coreMass = p.he_core_mass + p.c_core_mass + p.o_core_mass + p.si_core_mass + p.fe_core_mass
    # change from Msuns to grams
    coreMass = coreMass*1.989e33

    radius = p.radius
    radius = radius*69.551e9

    # setting up constants
    r2 = getR2(m2)
    m2 = m2*1.989e33 # units
    r2 = r2*69.551e9 # units
    rshred = r2 * (2*coreMass/m2)**(1/3)

    # make all the array sizes the same
    radius = radius[:-1]

    # look through everything in the radius array and compare it to rshred
    # when the value is <= rshred, save that index
    # chop the integrand and radius arrays at that index
    i = 0
    for x in radius:
        if x > rshred:
            i+=1

    radius = radius[100:i] # int first
    tInspiral = tInspiral[100:i] # int first
    
    radius = np.flip(radius)

    point = plt.plot(radius[0], tInspiral[0], 'o')
    y = plt.getp(point[0], 'color')
    plt.loglog(radius, tInspiral, label=label, color=y, linestyle=':')